In [1]:
import os

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras import layers
from keras import Input

tf.__version__

2023-08-26 22:41:44.811139: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 22:41:44.950228: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 22:41:44.952460: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-26 22:41:46.351462: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


'2.13.0'

In [2]:
# TODO there is a numpy util to get files from within tgz's and zips
##!curl -s 'https://storage.googleapis.com/0x19f.com/media/kaggle-mlb-pitch-data-2015-2018.tgz' | tar xz

BASEBALL_FILES_BASE = os.getcwd() + "/kaggle-mlb-pitch-data-2015-2018/"
BASEBALL_FILES_BASE = BASEBALL_FILES_BASE + "2019_"  # just look at 2019 to speed things up


pitches = pd.read_csv(BASEBALL_FILES_BASE + 'pitches.csv')
atbats = pd.read_csv(BASEBALL_FILES_BASE + 'atbats.csv')

df = pd.merge(pitches, atbats, how='inner', on='ab_id')

df = df[[
    # situation stuff
    "ab_id", "inning", "top", "outs", "on_1b", "on_2b", "on_3b",

    # matchup
    "pitcher_id", "p_throws", "batter_id", "stand",

    # specific pitch state
    "pitch_num", "b_count", "s_count",
    # TODO need to add running total of pitches for the current pitcher, and maybe strike percentage
    # TODO add the previous n pitches before and their code outcome
    # 
    
    # this is the label we're trying to predict
    "code",
    "pitch_type",
    # TODO also make the label have the location. not just FF (fourseam fastball, but also high inside)
]]

# remove anything without values, hard to have defaults
# df = df.dropna(subset=['pitch_type', 'code'])
df = df.dropna()

## this is our current target label TODO make this categorical 
df["is_fastball"] = np.where(df["pitch_type"].isin(["FC","FF","FT",]), 1, 0)

# get appropriate types. 
# TODO prob can use a schema on the CSV read to get only the columns we're after, alias them to be more
#      useful, and cast to the right type
# TODO consider to_ordinal for cumulative things maybe? inning, outs, pitch_num, b_count, s_count

# TODO there is a numpy util to get files from within tgz's and zips
##!curl -s 'https://storage.googleapis.com/0x19f.com/media/kaggle-mlb-pitch-data-2015-2018.tgz' | tar xz

BASEBALL_FILES_BASE = os.getcwd() + "/kaggle-mlb-pitch-data-2015-2018/"
BASEBALL_FILES_BASE = BASEBALL_FILES_BASE + "2019_"  # just look at 2019 to speed things up


pitches = pd.read_csv(BASEBALL_FILES_BASE + 'pitches.csv')
atbats = pd.read_csv(BASEBALL_FILES_BASE + 'atbats.csv')

df = pd.merge(pitches, atbats, how='inner', on='ab_id')

df = df[[
    # situation stuff
    "ab_id", "inning", "top", "outs", "on_1b", "on_2b", "on_3b",

    # matchup
    "pitcher_id", "p_throws", "batter_id", "stand",

    # specific pitch state
    "pitch_num", "b_count", "s_count",
    # TODO need to add running total of pitches for the current pitcher, and maybe strike percentage
    # TODO add the previous n pitches before and their code outcome
    # 
    
    # this is the label we're trying to predict
    "code",
    "pitch_type",
    # TODO also make the label have the location. not just FF (fourseam fastball, but also high inside)
]]

# remove anything without values, hard to have defaults
# df = df.dropna(subset=['pitch_type', 'code'])
df = df.dropna()

## this is our current target label TODO make this categorical 
df["is_fastball"] = np.where(df["pitch_type"].isin(["FC","FF","FT",]), 1, 0)

# get appropriate types. 
# TODO prob can use a schema on the CSV read to get only the columns we're after, alias them to be more
#      useful, and cast to the right type
# TODO consider to_ordinal for cumulative things maybe? inning, outs, pitch_num, b_count, s_count


# TODO consider using ints and normalizing them instead of floats for innnigs etc

df = df.astype({
    'p_throws': 'string',
    'stand': 'string',
  })


# drop ab_id now that join has occured, not a useful feature
df = df.drop(['ab_id', "code","pitch_type"], axis=1)

# TODO remove this truncation when solid
df = df.truncate(after=50000)

print(df.info())
print(df.shape)


<class 'pandas.core.frame.DataFrame'>
Index: 49774 entries, 0 to 50000
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   inning       49774 non-null  float64
 1   top          49774 non-null  float64
 2   outs         49774 non-null  float64
 3   on_1b        49774 non-null  float64
 4   on_2b        49774 non-null  float64
 5   on_3b        49774 non-null  float64
 6   pitcher_id   49774 non-null  int64  
 7   p_throws     49774 non-null  string 
 8   batter_id    49774 non-null  int64  
 9   stand        49774 non-null  string 
 10  pitch_num    49774 non-null  float64
 11  b_count      49774 non-null  float64
 12  s_count      49774 non-null  float64
 13  is_fastball  49774 non-null  int64  
dtypes: float64(9), int64(3), string(2)
memory usage: 5.7 MB
None
(49774, 14)


In [3]:
pitch_features = df.copy()
pitch_labels = pitch_features.pop('is_fastball')

In [29]:

inputs = {}

for name, col in pitch_features.items():
  if str(col.dtype) == "string":
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

{'inning': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'inning')>,
 'top': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'top')>,
 'outs': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'outs')>,
 'on_1b': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'on_1b')>,
 'on_2b': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'on_2b')>,
 'on_3b': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'on_3b')>,
 'pitcher_id': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'pitcher_id')>,
 'p_throws': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'p_throws')>,
 'batter_id': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'batter_id')>,
 'stand': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'stand')>,
 'pitch_num': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'pitch_num')>,
 'b_count': <KerasTensor: shape=(None, 1) dtype=float32 (crea

In [31]:
numeric_inputs = {
  name:input for name,input in inputs.items() if input.dtype==tf.float32
}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(pitch_features[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

preprocessed_inputs = [all_numeric_inputs]

for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  print("processing %s" % name)
  
  lookup = layers.StringLookup(vocabulary=np.unique(pitch_features[name]))
  one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)


preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

pitch_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)


In [ ]:
# tf.keras.utils.plot_model(model = pitch_preprocessing , rankdir="LR", dpi=72, show_shapes=True)


In [ ]:
titanic_features_dict = {name: np.array(value) 
                         for name, value in pitch_features.items()}

features_dict = {name:values[:1] for name, values in titanic_features_dict.items()}


def gen_pitch_model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
  ])

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])
  return model

pitch_model = gen_pitch_model(pitch_preprocessing, inputs)

pitch_model.fit(x=titanic_features_dict, y=pitch_labels, epochs=50)



